In [1]:
# Imports
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
# Create a sqlalchemy engine to input database
def get_engine(dbname, host='localhost', port=5432, user='nihar'):
    connection_string = f'postgresql://{user}@{host}:{port}/{dbname}'
    engine = create_engine(connection_string)
    return engine

In [3]:
# Returns user database
def get_users(hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # Establish the connection
    engine = get_engine(dbname='users')

    # Query to select all users
    query = "SELECT * FROM users;"

    # Read query result directly into pandas DataFrame
    df = pd.read_sql_query(query, engine)

    # Close the connection and return
    return df

In [4]:
def execute_query(dbname, query):
    # Establish the connection
    engine = get_engine(dbname)
    
    # Create transaction
    with engine.connect() as connection:
        transaction = connection.begin() 
        try:
            # Execute the query and commit changes
            connection.execute(text(query))
            transaction.commit()
        except Exception as e: # Re-raise the exception
            transaction.rollback()  
            raise e 

In [5]:
def add_user(username, email, password_hash, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # SQL query to insert a new user
    query = f"INSERT INTO users (username, email, password_hash) VALUES ('{username}', '{email}', '{password_hash}');"
    execute_query('users', query)

In [6]:
def remove_user(username, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # SQL query to delete a user based on the username
    query = f"DELETE FROM users WHERE username = '{username}';"
    execute_query('users', query)

In [7]:
# Get DB example
df = get_users()
df

,username,email,password_hash
0,john_doe,john@example.com,hashed_password_123
1,jane_doe,jane@example.com,hashed_password_456


In [8]:
# Add user example
remove_user("chungus") # Ensure doesn't exist since we're asserting users are unique
add_user("chungus", "bingus@yahoo.co.in", "hashed_password_222")
df = get_users()
df

,username,email,password_hash
0,john_doe,john@example.com,hashed_password_123
1,jane_doe,jane@example.com,hashed_password_456
2,chungus,bingus@yahoo.co.in,hashed_password_222


In [9]:
# Remove user example
remove_user("chungus")
df = get_users()
df

,username,email,password_hash
0,john_doe,john@example.com,hashed_password_123
1,jane_doe,jane@example.com,hashed_password_456
